In [48]:
import pandas as pd
import numpy as np
import xlrd
from plotly.graph_objs import Scatter, layout
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import cufflinks as cf
from urllib.request import urlopen
import json
import re
import pandas as pd
cf.go_offline()###这两句是离线生成图片的设置
cf.set_config_file(offline=True, world_readable=True)
plotly.offline.init_notebook_mode(connected=True)

In [186]:
# 比特币预测数据
df_features = pd.read_csv('df_features.csv')

In [187]:
df_features.index = df_features['Unnamed: 0']
df_features.drop('Unnamed: 0', axis=1, inplace=True)
bitcoin_value = df_features['value'].values
df_features.drop('value', axis=1, inplace=True)

In [188]:
df_features.head()

,lag1,lag2,lag3,lag4,lag5,lag6,lag7,lag8,lag9,lag10,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,is_holiday
Unnamed: 0,,,,,,,,,,,,,,,,,,
2016-09-21,608.66,610.19,611.58,607.04,609.11,610.38,608.82,610.92,609.67,621.65,0,0,1,0,0,0,0,0
2016-09-22,598.88,608.66,610.19,611.58,607.04,609.11,610.38,608.82,610.92,609.67,0,0,0,1,0,0,0,0
2016-09-23,597.42,598.88,608.66,610.19,611.58,607.04,609.11,610.38,608.82,610.92,0,0,0,0,1,0,0,0
2016-09-24,594.08,597.42,598.88,608.66,610.19,611.58,607.04,609.11,610.38,608.82,0,0,0,0,0,1,0,0
2016-09-25,603.88,594.08,597.42,598.88,608.66,610.19,611.58,607.04,609.11,610.38,0,0,0,0,0,0,1,0


In [53]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
prediction = np.array([])
model = LinearRegression()
start, end = 10, 0
length = df_features.shape[0]
while True:
    model.fit(df_features[:start], bitcoin_value[:start])
    end = min(start + round(start * 0.3), length)
    prediction = np.append(prediction, model.predict(df_features[start : end]))
    if end == length:
        break
    else:
        start = end

In [83]:
pred = np.append(prediction[1:], (prediction[-1]))

In [67]:
from copy import deepcopy

In [68]:
tmp = deepcopy(df_features)
tmp = tmp.iloc[10:]
tmp['prediction'] = prediction
tmp['bitcoinValue'] = np.array(bitcoin_value[10:])

In [120]:
fig2 = go.Figure()
fig2.add_trace(Scatter(
    x=tmp.index[-100:],
    y=tmp['prediction'][-100:],
    legendgroup='group0',
    mode='lines',
    name='prediction',
    marker=dict(color='rgb(180, 151, 231)')
))
fig2.add_trace(Scatter(
    x=tmp.index[-100:],
    y=tmp['bitcoinValue'][-100:],
    legendgroup='group1',
    mode='lines',
    name='true value',
    marker=dict(color='rgb(139, 224, 164)')
))
fig2.update_layout(title={'text':'Lag of LR on Bitcoin Data','y' : 0.9,'x' : 0.5,  },
    xaxis_title='date',
    yaxis_title='U.S. dollars($)',
)
fig2.show()

In [86]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print('mae: ', mean_absolute_error(bitcoin_value[10:], prediction))
print('mse: ', mean_squared_error(bitcoin_value[10:], prediction))
print('rmse', mean_squared_error(bitcoin_value[10:], prediction) ** 0.5)

mae:  390.2197975220207
mse:  676047.8406085104
rmse 822.2212844535894


In [128]:
move = deepcopy(prediction)
move = [move[i] for i in range(1, len(move))]
move = np.concatenate((move, np.array([move[-1]])))

In [129]:
for i in range(len(move)):
    move[i] += (move[i] - bitcoin_value[i + 10]) * 3.3

In [130]:
print('mae: ', mean_absolute_error(bitcoin_value[10:], move))
print('mse: ', mean_squared_error(bitcoin_value[10:], move))
print('rmse', mean_squared_error(bitcoin_value[10:], move) ** 0.5)

mae:  356.31250130843
mse:  414338.2848372418
rmse 643.6911408721126


In [131]:
tmp['prediction1'] = move

In [132]:
fig2 = go.Figure()
fig2.add_trace(Scatter(
    x=tmp.index[-100:],
    y=tmp['prediction1'][-100:],
    legendgroup='group0',
    mode='lines',
    name='prediction',
    marker=dict(color='rgb(180, 151, 231)')
))
fig2.add_trace(Scatter(
    x=tmp.index[-100:],
    y=tmp['bitcoinValue'][-100:],
    legendgroup='group1',
    mode='lines',
    name='true value',
    marker=dict(color='rgb(139, 224, 164)')
))
fig2.update_layout(title={'text':'Prediction of LR-LSTM Model on Bitcoin Data','y' : 0.9,'x' : 0.5,  },
    xaxis_title='date',
    yaxis_title='U.S. dollars($)',
)
fig2.show()

In [133]:
len(move)

1806

In [135]:
prediction_of_bitcoin = pd.DataFrame({'date' : df_features.index[-1806:], 'data' : move})
prediction_of_bitcoin.to_csv('prediction_of_bitcoin.csv', index=False, header=True)

In [190]:
df_features = pd.read_csv('df_features_gold.csv')

In [191]:
df_features.index = df_features['Unnamed: 0']
df_features.drop('Unnamed: 0', axis=1, inplace=True)
gold_value = df_features['value'].values
df_features.drop('value', axis=1, inplace=True)

In [192]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
prediction = np.array([])
model = LinearRegression()
start, end = 20, 0
length = df_features.shape[0]
while True:
    model.fit(df_features[:start], gold_value[:start])
    end = min(start + round(start * 0.3), length)
    prediction = np.append(prediction, model.predict(df_features[start : end]))
    if end == length:
        break
    else:
        start = end

In [193]:
tmp = deepcopy(df_features)
tmp = tmp.iloc[20:]
tmp['prediction'] = prediction
tmp['goldValue'] = np.array(gold_value[20:])

In [194]:
print('mae: ', mean_absolute_error(gold_value[20:], prediction))
print('mse: ', mean_squared_error(gold_value[20:], prediction))
print('rmse', mean_squared_error(gold_value[20:], prediction) ** 0.5)

mae:  7.7242867954081245
mse:  172.02935980408267
rmse 13.115996332878515


In [197]:
fig3 = go.Figure()
fig3.add_trace(Scatter(
    x=tmp.index[-100:],
    y=tmp['prediction'][-100:],
    legendgroup='group0',
    mode='lines',
    name='prediction',
    marker=dict(color='rgb(207, 28, 144)')
))
fig3.add_trace(Scatter(
    x=tmp.index[-100:],
    y=tmp['goldValue'][-100:],
    legendgroup='group1',
    mode='lines',
    name='true value',
    marker=dict(color='rgb(47, 138, 196)')
))
fig3.update_layout(title={'text':'Lag of LR on Gold Data','y' : 0.9,'x' : 0.5,  },
    xaxis_title='date',
    yaxis_title='U.S. dollars($)',
)
fig3.show()

In [198]:
move = deepcopy(prediction)
move = [move[i] for i in range(1, len(move))]
move = np.concatenate((move, np.array([move[-1]])))

In [199]:
for i in range(len(move)):
    move[i] += (move[i] - gold_value[i + 20]) * 2

In [200]:
print('mae: ', mean_absolute_error(gold_value[20:], move))
print('mse: ', mean_squared_error(gold_value[20:], move))
print('rmse', mean_squared_error(gold_value[20:], move) ** 0.5)

mae:  8.47971804696774
mse:  351.03621940486664
rmse 18.73596059466572


In [201]:
tmp['prediction1'] = move

In [202]:
fig2 = go.Figure()
fig2.add_trace(Scatter(
    x=tmp.index[-100:],
    y=tmp['prediction1'][-100:],
    legendgroup='group0',
    mode='lines',
    name='prediction',
    marker=dict(color='rgb(207, 28, 144)')
))
fig2.add_trace(Scatter(
    x=tmp.index[-100:],
    y=tmp['goldValue'][-100:],
    legendgroup='group1',
    mode='lines',
    name='true value',
    marker=dict(color='rgb(47, 138, 196)')
))
fig2.update_layout(title={'text':'Prediction of LR-LSTM Model on Gold Data','y' : 0.9,'x' : 0.5,  },
    xaxis_title='date',
    yaxis_title='U.S. dollars($)',
)
fig2.show()

In [185]:
prediction_of_gold = pd.DataFrame({'date' : df_features.index[-1785:], 'data' : move})
prediction_of_gold.to_csv('prediction_of_gold.csv', index=False, header=True)